# Network Analysis on the Embeddings of Span Subsentences Generatedy by GEM
The analyses in this notebook use the results generated by the analyses on SPARK in the notebook sent-subsent-embs-graph-analysis-SPARK.ipynb



In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

%matplotlib inline

## Set a Timer

In [ ]:
# For timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start

# Load in the Embeddings of Span Subsentences Generatedy by GEM

In [ ]:
import pickle

In [ ]:
import torch
embs_span = torch.load('NYT_train_gem_embs_span.pt')

In [ ]:
embs_span.shape

# Compute the Pair-Wise Distance

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
# Test
with elapsed_timer() as elapsed:
    embs_span_pdist_5 = pdist(embs_span[:5])
    duration = '%.1f' % elapsed()
print(" Computing the distances between {} points takes {} seconds".format(embs_span.shape[0], duration))  

In [ ]:
#np.save('NYT_train_gem_embs_pdist.np', embs_pdist)

In [ ]:
with elapsed_timer() as elapsed:
    embs_span_pdist = pdist(embs_span)
    duration = '%.1f' % elapsed()
print(" Computing the distances between {} points takes {} seconds".format(embs_span.shape[0], duration))  

In [ ]:
np.save('NYT_train_gem_embs_span_pdist.np', embs_span_pdist)

In [ ]:
###############################

# Compute the mean distance within groups

In [ ]:
embs_span_contains = embs_span[train_df.relIdx == 3]

In [ ]:
embs_span_contains.shape

In [ ]:
train_df.groupby('relIdx').size().reset_index(name='count').sort_values('count', ascending=False)

In [ ]:
with elapsed_timer() as elapsed:
    embs_span_contains_pdist = pdist(embs_span_contains)
    duration = '%.1f' % elapsed()
print(" Computing the distances between {} points takes {} seconds".format(embs_span_contains.shape[0], duration))

In [ ]:
np.histogram(embs_span_contains_pdist, bins=20, density=False)

In [ ]:
embs_span_contains_pdist.mean()

In [ ]:
embs_span_4 = embs_span[train_df.relIdx == 1]

In [ ]:
with elapsed_timer() as elapsed:
    embs_span_4_pdist = pdist(embs_span_4)
    duration = '%.1f' % elapsed()
print(" Computing the distances between {} points takes {} seconds".format(embs_span_4.shape[0], duration))

In [ ]:
embs_span_4_pdist.mean()

# Compute Histogram of the pdist

In [ ]:
#embs_pdist = np.load('NYT_train_gem_embs_pdist.np.npy')

In [ ]:
embs_span_pdist.shape

In [ ]:
np.histogram(embs_span_pdist, bins=20, density=False)

In [ ]:
count = [    4429938,  189362009, 1400106528, 1931831025, 1435444474,
         761734433,  329423883,  120725634,   37503175,   11299382,
           4121778,    1319587,     609633,     244287,     157528,
             24952,       1772,        177,         42,          8]
bin = [  0.        ,  4.91796632,  9.83593265, 14.75389897, 19.67186529,
        24.58983162, 29.50779794, 34.42576426, 39.34373059, 44.26169691,
        49.17966323, 54.09762956, 59.01559588, 63.9335622 , 68.85152853,
        73.76949485, 78.68746117, 83.6054275 , 88.52339382, 93.44136014,
        98.35932647]

In [ ]:
import math
midpoints = []
for i in range(len(bin)-1):
    midpoints.append(math.floor((bin[i+1]-bin[i]) //2 + bin[i]))

In [ ]:
midpoints

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(midpoints, count, width=10)
plt.show()

In [ ]:
plt.figure(figsize=(15,9))
plt.bar(midpoints, count, width=5)
plt.xticks(midpoints)
plt.xlabel('Euclidean Distance')
plt.ylabel('Counts (10^10)')
plt.savefig("GEM_embs_span_pdist_hist.pdf")

In [ ]:
hist_df = pd.DataFrame({'count':count, 'bin_right':bin[1:]})
hist_df.head()

In [ ]:
hist_df.to_csv('GEM_embs_span_pdist_hist.csv', index=False)

# Compute Mean, Max, Min

In [ ]:
embs_span_pdist.mean()

In [ ]:
embs_span_pdist.max()

In [ ]:
embs_span_pdist.min()

# Compute a Graph by dist < mean

In [ ]:
m = 111610
(m * m) /2 - m /2 # 6228340245

In [ ]:
i = 111608
j = 111609
m * i + j - ((i+2)*(i+1))//2

In [ ]:
embs_span_pdist[m * i + j - ((i+2)*(i+1))//2]

In [ ]:
i = 0
j = 0

In [ ]:
embs_span_pdist_test = embs_span_pdist[20:50]

In [ ]:
np.where(embs_span_pdist_test < 19)

In [ ]:
# from upper triangular index back to matrix indices
from math import floor, sqrt, ceil
def coor_to_idx(n, i, j):
    return i*(2*n-i+1)//2+j-i
def idx_to_coor(n, k):
    i = floor((-sqrt((2*n+1)*(2*n+1)-8*k)+2*n+1)/2)
    j = k + i - i*(2*n-i+1)//2
    return i, j

def coor_to_idx_shift(n, i, j):
    return n * i + j - ((i + 1) * (i + 2)) //2
    
def idx_to_coor_shift(n, k):
    relk = k
    
    i = floor((-sqrt((2*n+1)*(2*n+1)-8*k)+2*n+1)/2)

    #translate row number shifted for diagnol
    k = k + i + 1
    
    for p in range(k, n * n):
        i = floor((-sqrt((2 * n + 1) * (2 * n + 1) - 8 * p) + 2 * n + 1) / 2)
        if(relk == p - i - 1):
            j = p + i - i*(2*n-i+1)//2
            return i, j
    
    return -1, -1

In [ ]:
for k in range(0, 10):
    print(idx_to_coor(4, k))

In [ ]:
embs_span_graph_edges = np.nonzero(embs_span_pdist < 19)

In [ ]:
embs_span_graph_edges = np.nonzero(embs_span_pdist < 9)

In [ ]:
def idx_to_coor_vec(k):
    return idx_to_coor_shift(111610, k)

In [ ]:
idx_vec = np.vectorize(idx_to_coor_vec)

In [ ]:
np.save("GEM_embs_span_graph_edges_dist_less_9.npy", embs_span_graph_edges[0])

In [ ]:
edges = idx_vec(embs_span_graph_edges[0])

In [ ]:
np.save('GEM_span_graph_edges_end1.npy', edges[0])

In [ ]:
np.save('GEM_span_graph_edges_end2.npy', edges[1])

# Make Graph Edge List

In [ ]:
end1 = np.load('GEM_span_graph_edges_end1.npy')

In [ ]:
end1.shape

In [ ]:
end2 = np.load('GEM_span_graph_edges_end2.npy')

In [ ]:
end2.shape

In [ ]:
end2.max()

In [ ]:
all_span_edges_df = pd.DataFrame({'end1': end1, 'end2':end2})

In [ ]:
all_span_edges_df.head()

In [ ]:
all_span_edges_df.to_csv("GEM_all_span_edges.csv", header=False, index=False)

In [ ]:
all_span_edges_df.shape

In [ ]:
all_span_edges_df = pd.read_csv("GEM_all_span_edges.csv")

In [ ]:
all_span_edges_df.shape

In [ ]:
# save all edges as nodeId nodeId
all_span_edges_df.to_csv("GEM_all_span_edges_space.csv", sep=" ", header=False, index=False)

# Degree and Size Analysis

In [ ]:
degrees = pd.read_csv("span_degrees/span_degrees.csv")

In [ ]:
degrees.head()

In [ ]:
degrees['degree'].unique()

In [ ]:
degrees[degrees.degree > 100]

In [ ]:
degrees.groupby('degree').count().sort_values('degree', ascending=False)

In [ ]:
degrees[degrees.degree >= 31620].shape

In [ ]:
degrees.groupby('degree').size().reset_index(name='counts').sort_values('degree', ascending=False)

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 14}

matplotlib.rc('font', **font)

In [ ]:
ax = plt.gca()
degrees.degree.hist(figsize=(12,6))
ax.grid(False)
ax.set_xlabel("Degree")
ax.set_ylabel("Number of Vertices")
ax.set_yticks(np.arange(0, 90000, 10000))
plt.savefig("figures/GEM_span_degrees.pdf")

In [ ]:
# check node ids who have largest degree
degrees[degrees.degree==31620]

In [ ]:
bdegrees = degrees[degrees['degree'] == 31620]['id']

In [ ]:
train_df[(train_df.index.isin(bdegrees)) & (train_df.relIdx == 3)]

In [ ]:
train_df.iloc[39803].sentText

In [ ]:
train_df.iloc[40558].sentText

# Load the NYT train Data

In [ ]:
train_df = pd.read_csv("NYT_train_df_nn.csv", index_col=0)

In [ ]:
train_df.columns

In [ ]:
for idx in degrees[degrees.degree.isin(bdegrees)].id:
    print(train_df.iloc[idx].sentText, train_df.iloc[idx].relIdx)

In [ ]:
train_df['relation'].unique(), train_df['relIdx'].unique()

## Relation Distribution

In [ ]:
train_df.groupby('relIdx').size().reset_index(name='count').sort_values('count', ascending=False)

# Connected Components

In [ ]:
cc = pd.read_csv('span_cc/span_connectedComponents.csv')

In [ ]:
cc.head()

In [ ]:
cc.shape

In [ ]:
ccsizes = cc.groupby('component').size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
ccsizes.head()

In [ ]:
ccsizes[1:].hist('counts')

In [ ]:
ccsizes.head()

In [ ]:
import seaborn as sns
sns.distplot(ccsizes[1:]['counts'])

In [ ]:
# Largest cc
lgst_cc_ids = cc[cc.component == 9].id

In [ ]:
lgst_cc_ids.shape

In [ ]:
lgst_cc_ids.to_csv("Span_Largest_Connected_Component_IDs.csv", index=False)

In [ ]:
lgst_cc_ids.head()

In [ ]:
all_span_edges = pd.read_csv('GEM_all_span_edges.csv')

In [ ]:
all_span_edges.columns = ['end1', 'end2']
all_span_edges.head()

In [ ]:
all_span_edges.end2.nunique()

In [ ]:
len(set(all_span_edges.end1).union(set(all_span_edges.end2)))

In [ ]:
all_span_edges.shape

In [ ]:
all_span_lcc_edges = all_span_edges[(all_span_edges['end1'].isin(lgst_cc_ids)) | (all_span_edges['end2'].isin(lgst_cc_ids))]

In [ ]:
all_span_lcc_edges.shape

In [ ]:
all_span_lcc_edges.to_csv("GEM_span_largest_CC_edges.csv", sep=' ', header=False, index=False)

# Numbers Edges in Each CC

In [ ]:
cc.component.nunique(), cc.shape

In [ ]:
cc_num = cc.component.unique()

In [ ]:
cc_num

In [ ]:
%%time
#cc_nums = []
#cc_sizes = []
#i = 1
#for n in cc_num:
#    print(i)
#    i = i + 1
#    cc_nums.append(n)
#    cc_ids = cc[cc.component == n].id
#    cc_edges = all_span_edges[(all_span_edges['end1'].isin(cc_ids)) | (all_span_edges['end2'].isin(cc_ids))]
#    cc_sizes.append(cc_edges.shape[0])

In [ ]:
len(cc_nums),len(cc_sizes)

In [ ]:
cc_num_sizes_df = pd.DataFrame({'cc':cc_num, 'size':cc_sizes})

In [ ]:
cc_num_sizes_df.shape

In [ ]:
cc_num_sizes_df.to_csv('GEM_span_graph_cc_sizes.csv', index=False)

In [ ]:
cc_size_counts = cc_num_sizes_df.groupby('size').size().reset_index(name='counts').sort_values('counts', ascending=False)

In [ ]:
cc_size_counts.shape

In [ ]:
cc_size_counts.tail()

In [ ]:
cc_num_sizes_df.groupby('size').size().reset_index(name='counts').sort_values('size', ascending=False)

In [ ]:
plt.hist(cc_size_counts[:-1]['size'])

In [ ]:
plt.bar(cc_size_counts[:-1]['size'], cc_size_counts[:-1]['counts'])

# Largest Connected Component Degrees

In [ ]:
lccdegrees = pd.read_csv("span_lcc_degrees.csv")

In [ ]:
lccdegrees.head()

In [ ]:
lccdegrees.shape

In [ ]:
lccdegrees.groupby('degree').size().reset_index(name='counts').sort_values('degree', ascending=False)

In [ ]:
lccdegrees[lccdegrees.degree > 10000]

# Page Rank Analysis

In [ ]:
ranks = pd.read_csv('span_lcc_page_ranks.csv')

In [ ]:
ranks.shape

In [ ]:
ranks.head()

In [ ]:
ranks.pagerank.min(), ranks.pagerank.max(), ranks.pagerank.mean()

In [ ]:
cuts = pd.cut(ranks.pagerank, 20)

In [ ]:
cuts.value_counts()

In [ ]:
ranks[ranks.pagerank > 1330]

In [ ]:
train_df.iloc[111605]

In [ ]:
lccdegrees[lccdegrees.id == 111605]

# Shortest Paths

In [ ]:
import json

In [ ]:
with open('span_lcc_shortest_paths_randomPoint1Percent.json', 'r') as f:
    p11 = json.loads(f.readline())

In [ ]:
for it in p11['distances'].keys():
    print(p11['id'], it, p11['distances'][it])

In [ ]:
p11['distances']

In [ ]:
p11

In [ ]:
files = ['span_lcc_shortest_paths_randomPoint1Percent.json', 'span_lcc_shortest_paths_randomPoint2Percent.json', \
         'span_lcc_shortest_paths_randomPoint3Percent.json', 'span_lcc_shortest_paths_randomPoint4Percent.json', \
         'span_lcc_shortest_paths_randomPoint5Percent.json', 'span_lcc_shortest_paths_randomPoint6Percent.json', \
         'span_lcc_shortest_paths_randomPoint7Percent.json', 'span_lcc_shortest_paths_randomPoint8Percent.json', 
         'span_lcc_shortest_paths_randomPoint9Percent.json', 'span_lcc_shortest_paths_randomPoint10Percent.json']

In [ ]:
src = []
tgt = []
dist = []
for file in files:
    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            p = json.loads(line)
            for it in p['distances']:
                src.append(p['id'])
                tgt.append(it)
                dist.append(p['distances'][it])

In [ ]:
dists = pd.DataFrame({'src':src, 'tgt':tgt, 'dist':dist})

In [ ]:
dists.shape

In [ ]:
dists.to_csv('span_lcc_shortestpaths_estimates_1percent.csv', index=False)

In [ ]:
dists.groupby('dist').size() / 1253830

In [ ]:
dists.groupby('dist').size()

In [ ]:
dists.dist.mean()

# Largest Degrees in Largest Connected Components

In [ ]:
lccdegrees.groupby(['degree', 'id']).size().reset_index(name='counts').sort_values('degree', ascending=False).head(10)

In [ ]:
train_df.iloc[40558].relation

In [ ]:
all_span_lcc_edges.columns

In [ ]:
all_span_lcc_edges.shape

In [ ]:
all_span_lcc_edges.head()

In [ ]:
all_span_lcc_edges[(all_span_lcc_edges.end1 == 40558) & (all_span_lcc_edges.end2 == 39803)]

In [ ]:
all_span_lcc_edges[(all_span_lcc_edges.end1 == 39803) & (all_span_lcc_edges.end2 == 40558)]

In [ ]:
edges_39803 = all_span_lcc_edges[(all_span_lcc_edges.end1 == 39803) | (all_span_lcc_edges.end2 == 39803)]

In [ ]:
edges_39803.shape

In [ ]:
edges_39803_20 = edges_39803.sample(20)

In [ ]:
edges_39803_20.end1.unique(), edges_39803_20.end2.unique()

In [ ]:
edges_39803_nbs = [2666, 17363,  6668, 39803, 24953, 27312, 17125, 21373, 24348, \
                   89088,  45049, 108051,  50079,  95975,  65258,
                   58038, 105193,  43409, 40558]

In [ ]:
graph_39803 = all_span_lcc_edges[(all_span_lcc_edges.end1.isin(edges_39803_nbs)) & (all_span_lcc_edges.end2.isin(edges_39803_nbs))]

In [ ]:
graph_39803.shape

In [ ]:
graph_39803.to_csv('span_graph_39803.csv', header=False, index=False)

In [ ]:
import networkx as nx

In [ ]:
with open("span_graph_39803.csv", "rb") as fh:
    G = nx.read_edgelist(fh, delimiter=",")

In [ ]:
#plt.figure(figsize=(15,12))
nx.draw(G, with_labels=True)

In [ ]:
train_df.iloc[edges_39803_nbs[4]].relation

In [ ]:
import re
pat = r"/.*/"

vid_vlabel = {}
for idx in edges_39803_nbs:
    #print(id)
    vid_vlabel[str(idx)] = re.sub(pat, "", train_df.iloc[idx].relation)

In [ ]:
vid_vlabel

In [ ]:
for idx in edges_39803_nbs:
    G.nodes[str(idx)]['name'] = vid_vlabel[str(idx)]

## Graph of Second Node with Largest Degree

In [ ]:
edges_40558 = all_span_lcc_edges[(all_span_lcc_edges.end1 == 40558) | (all_span_lcc_edges.end2 == 40558)]

In [ ]:
edges_40558.shape

In [ ]:
edges_40558_20 = edges_40558.sample(30)

In [ ]:
edges_40558_20.end1.unique(), edges_40558_20.end2.unique()

In [ ]:
edges_40558_nbs = [28386, 3599, 10422, 37935, 40558, 17892,  6146, 22652,\
        36086, 35616,  22735, 28983, \
        92434, 111539, 62650, 105018, \
         51379,  50273,  86059,  57074, 104020,  69405, \
         72870,  63391] 

In [ ]:
graph_40558 = all_span_lcc_edges[(all_span_lcc_edges.end1.isin(edges_40558_nbs)) & (all_span_lcc_edges.end2.isin(edges_40558_nbs))]

In [ ]:
graph_40558.shape

In [ ]:
graph_40558.to_csv('span_graph_40558.csv', header=False, index=False)

In [ ]:
# merge graph_39803 and graph_40558
graph_39803_40558 = pd.concat([graph_39803, graph_40558])

In [ ]:
graph_39803_40558.to_csv('span_graph_39803_40558.csv', header=False, index=False)

In [ ]:
with open("span_graph_40558.csv", "rb") as fh:
    G40558 = nx.read_edgelist(fh, delimiter=",")

In [ ]:
#plt.figure(figsize=(15,12))
nx.draw(G40558, with_labels=True)

In [ ]:
train_df.iloc[edges_40558_nbs[4]].relation

In [ ]:
import re
pat = r"/.*/"

vid_vlabel = {}
for idx in edges_40558_nbs:
    #print(id)
    vid_vlabel[str(idx)] = re.sub(pat, "", train_df.iloc[idx].relation)

In [ ]:
vid_vlabel

In [ ]:
with open('span_graph_39803_40558.csv', 'rb') as fh:
    Gboth = nx.read_edgelist(fh, delimiter=",")

In [ ]:
Gboth.nodes

In [ ]:
Gboth.number_of_edges()

In [ ]:
edges_both = []
for i in [edges_39803_nbs, edges_40558_nbs]:
    for j in i:
        if(j not in edges_both):
            edges_both.append(j)

In [ ]:
import re
pat = r"/.*/"

vid_vlabel = {}
for idx in edges_both:
    #print(id)
    vid_vlabel[str(idx)] = re.sub(pat, "", train_df.iloc[idx].relation)

In [ ]:
for idx in edges_both:
    Gboth.nodes[str(idx)]['name'] = vid_vlabel[str(idx)]

In [ ]:
nx.write_gml(Gboth, 'span_graph_39803_40558.gml')

In [ ]:
gg = nx.read_gml('span_graph_39803_40558.gml')

In [ ]:
gg.nodes

In [ ]:
train_df.iloc[39803]

In [ ]:
train_df.iloc[40558]

# Relation Distribution in the LCC

In [ ]:
lgst_cc_ids.shape

In [ ]:
lgst_cc_ids.head()

In [ ]:
train_df_lcc = train_df.iloc[lgst_cc_ids]

In [ ]:
train_df_lcc.shape

In [ ]:
train_df_relgrp = train_df.groupby('relIdx').size().reset_index(name='count').sort_values('count', ascending=False)

In [ ]:
train_df_lcc_relgrp = train_df_lcc.groupby('relIdx').size().reset_index(name='count').sort_values('count', ascending=False)

In [ ]:
plt.figure(figsize=(15, 12))
plt.pie(train_df_relgrp['count'] / train_df.shape[0], labels=train_df_relgrp.relIdx)

In [ ]:
idx_rel = {1:'nationality', 2:'capital', \
        3:'contains',\
        4:'other',\
        5:'other', 6:'place_of_birth',\
        7:'place_lived',\
        8:'administrative_divisions',\
        9:'country',\
        10:'company',\
        11:'neighborhood_of',\
        '12':'place_founded', 13:'other',\
        14:'other',\
        15:'other',\
        16:'other',
        17:'other', 18:'other',\
        19:'other', 20:'other',\
        21:'other',\
        22:'other',\
        23:'other', 24:'other'}

In [ ]:
train_df_relgrp['rel'] = train_df_relgrp.relIdx.map(idx_rel)

In [ ]:
pd.DataFrame([[5107, 'other']], columns=['count', 'rel'])

In [ ]:
train_df_relgrp_other = pd.concat([train_df_relgrp.reset_index().iloc[0:9][['count', 'rel']], \
          pd.DataFrame([[5107, 'other']], columns=['count', 'rel'])])

In [ ]:
plt.figure(figsize=(15, 12))
plt.pie(train_df_relgrp_other['count'] / train_df.shape[0], labels=train_df_relgrp_other.rel)

In [ ]:
plt.figure(figsize=(15, 12))
plt.pie(train_df_lcc_relgrp['count'] / train_df_lcc.shape[0], labels=train_df_lcc_relgrp.relIdx)

In [ ]:
train_df_lcc_relgrp['count'].sum()

In [ ]:
lcc_contains_density = pd.DataFrame({'In_original_dataset': [0.48], 'In_largest_CC_sentences': [0.50], 'In_largest_CC_span': [0.55]})

In [ ]:
plt.bar([1,2,3], [0.48, 0.50, 0.55])

# Sentence Example

In [ ]:
lens = train_df.sentText.apply(lambda s: len(s.split()))

In [ ]:
train_df[lens > 20]

In [ ]:
text_10 = train_df[lens > 20].sentText

# All KLs

In [ ]:
allkls = pd.read_csv("All-KLs.csv", index_col=0)

In [ ]:
allkls.head()

In [ ]:
allkls.index

In [ ]:
labs = ['sentence', 'span', 'spanBA1', 'spanBA2', 'spanBA3',
       'spanBA4', 'spanBA5', 'spanBA6', 'spanBA7',
       'spanBA8', 'spanBA9', 'spanBA10', 'spanBA15',
       'spanBA20', 'surroundings1', 'surroundings2', 'surroundings3',
       'surroundings4', 'surroundings5', 'surroundings6', 'surroundings7',
       'surroundings8', 'surroundings9', 'surroundings10', 'surroundings15',
       'surroundings20']

In [ ]:
ticks = np.arange(len(labs))

In [ ]:
plt.figure(figsize=(15, 9))
allkls.plot(figsize=(15,9), legend=False)
plt.xticks(ticks, labs, rotation=30)
plt.ylabel('Spatial Histogram')
plt.text(1,4.2,'GloVe-GEM')
plt.text(6,3.5,'Skipthought')
plt.text(8,2.5,'GloVe-DCT')
plt.text(2,1.8,'GloVe-Mean')
plt.text(8,1.1,'SentenceBERT')
plt.text(10,1.5,'Quickthought')
plt.text(6,1.5,'LASER')
plt.text(5.5,1.8,'InferSentv1')
plt.text(19,1.4,'InferSentv2')
plt.savefig("figures/all-spatHist.pdf")

In [ ]:
allkls.iloc[[1,2,3,4,5,6,7,8,9,10,11,12,13,0], :].plot(figsize=(15,9))

In [ ]:
allkls.iloc[[0,14,15,16,17,18,19,20,21,22,23,24,25],:].plot(figsize=(15,9))

In [ ]:
allkls_std = pd.read_csv("all-KLs-std.csv", index_col=0)

In [ ]:
allkls_std.min().min(), allkls_std.max().max(), allkls_std.mean().mean(0)

In [ ]:
allkls_std.plot(figsize=(15,9))

In [ ]:
allclseval_sbert = pd.read_csv("all-cls-eval-sbert.csv", index_col=0)

In [ ]:
allclseval_sbert.plot(figsize=(15,9))

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 14}

matplotlib.rc('font', **font)

In [ ]:
allclseval_sbert.iloc[[0,1,2,3,13, 14, 15, 24, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(3.5,0.06, 'Sentence')
plt.text(2.5,0.2, 'surroundings1')
plt.text(3.5,0.16, 'span')
plt.text(5.5,0.09, 'surroundings15')
plt.text(3.7,0.09, 'spanBA20')
plt.savefig("figures/all_cls_eval_sbert.pdf")

In [ ]:
allclseval_laser = pd.read_csv("all-cls-eval-laser.csv", index_col=0)

In [ ]:
allclseval_laser.iloc[[0,1,2,3,13, 14, 15, 24, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(3.5,0.08, 'Sentence')
plt.text(2.5,0.25, 'surroundings1')
plt.text(3.5,0.22, 'span')
plt.text(5.5,0.11, 'surroundings15')
plt.text(3.7,0.11, 'spanBA20')
plt.savefig("figures/all_cls_eval_laser.pdf")

In [ ]:
allclseval_infersentv2 = pd.read_csv("all-cls-eval-infersentv2.csv", index_col=0)

In [ ]:
allclseval_infersentv2.iloc[[0,1,2,3,13, 14, 15, 24, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(3.5,0.09, 'Sentence')
plt.text(2.5,0.30, 'surroundings1')
plt.text(3.5,0.22, 'span')
plt.text(5.5,0.12, 'surroundings15')
plt.text(4.7,0.09, 'spanBA20')
plt.savefig("figures/all_cls_eval_infersentv2.pdf")

In [ ]:
allclseval_infersentv1 = pd.read_csv("all-cls-eval-infersentv1.csv", index_col=0)

In [ ]:
allclseval_infersentv1.iloc[[0,1,2,3,13, 14, 15, 24, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(6.2,0.10, 'Sentence')
plt.text(2.5,0.32, 'surroundings1')
plt.text(3.5,0.23, 'span')
plt.text(2.7,0.14, 'spanBA20')
plt.text(3.0,0.07, 'surroundings20')
plt.savefig("figures/all_cls_eval_infersentv1.pdf")

In [ ]:
allclseval_quickthought= pd.read_csv("all-cls-eval-quickthought.csv", index_col=0)

In [ ]:
allclseval_quickthought.iloc[[0,1,13, 14, 15, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(6.0,0.10, 'surroundings2')
plt.text(5.8,0.15, 'surroundings20')
plt.text(3.0,0.14, 'surroundings1')
plt.text(2.5,0.23, 'span')
plt.text(3.2,.04, 'spanBA20')
plt.text(3.0,0.07, 'Sentence')
plt.savefig("figures/all_cls_eval_quickthought.pdf")

In [ ]:
allclseval_skipthought= pd.read_csv("all-cls-eval-skipthought.csv", index_col=0)

In [ ]:
allclseval_skipthought.iloc[[0,1,2,13, 14, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(6.0,0.10, 'surroundings1')
plt.text(5.0,0.05, 'spanBA1')
plt.text(3.0,0.13, 'surroundings20')
plt.text(3,0.22, 'span')
plt.text(3.2,.04, 'Sentence')
plt.text(5.0,0.01, 'spanBA20')
plt.savefig("figures/all_cls_eval_skipthought.pdf")

In [ ]:
allclseval_glove_gem= pd.read_csv("all-cls-eval-glove-gem.csv", index_col=0)

In [ ]:
allclseval_glove_gem.iloc[[0,1,2,13, 14, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(3.5,0.08, 'spanBA1')
plt.text(5.0,0.05, 'spanBA20')
plt.text(2.2,0.005, 'surroundings20')
plt.text(3,0.1, 'span')
plt.text(2.9,.06, 'Sentence')
plt.text(5.0,0.012, 'surroundings1')
plt.savefig("figures/all_cls_eval_glove_gem.pdf")

In [ ]:
allclseval_glove_dct= pd.read_csv("all-cls-eval-glove-dct.csv", index_col=0)

In [ ]:
allclseval_glove_dct.iloc[[0,1,2,13, 14, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(3.5,0.21, 'span')
plt.text(4,0.04, 'suroundings1')
plt.text(3.2,0.001, 'Sentence')
plt.text(3,0.11, 'spanBA1')
plt.text(2.7,.06, 'surroundings20')
plt.text(5.0,0.012, 'spanBA20')
plt.savefig("figures/all_cls_eval_glove_dct.pdf")

In [ ]:
allclseval_glove_mean= pd.read_csv("all-cls-eval-glove-mean.csv", index_col=0)

In [ ]:
allclseval_glove_mean.iloc[[0,1,2,13, 14, 25],:].T.plot(figsize=(9,6), legend=False)
plt.xticks(rotation=20)
plt.text(3.5,0.21, 'span')
plt.text(2.8,0.3, 'suroundings1')
plt.text(3.2,0.001, 'Sentence')
plt.text(3.8,.25, 'spanBA1')
plt.text(2.7,.06, 'surroundings20')
plt.text(2.8,0.15, 'spanBA20')
plt.savefig("figures/all_cls_eval_glove_mean.pdf")